In [ ]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'chinapeace_gov_cn'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
publish_source = 'chinapeace.gov.cn'
country = 'China'
language = 'Chinese'
images_path = []

headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
r = requests.get('http://www.chinapeace.gov.cn/', headers=headers, timeout=60).text

home_page = c.scrap('<div class="nav-new2020">(.*?)<div class="incabb">',r)

for _ in home_page.split('<li><a href=')[12:13]: 
    
    cat_url = c.scrap('"(.*?)"', _)   
    

    if  not cat_url.startswith('http://www.chinapeace.gov.cn'):# start with condition
        cat_url = 'http://www.chinapeace.gov.cn' +  cat_url
        
    logger.info(f'Fetching cat url  {cat_url}\n')

    cat_page = requests.get(cat_url, headers=headers, timeout=120).text

    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) # writes error message with error code
        unable_to_fetch_cat_url += 1
        continue    
       
    cat_page=c.scrap('<div class="list_box_left">(.*?)</script>',cat_page)#'<li><a href='or<h2><a href=
    if '<li><a href=' in cat_page:     # if li present,use this
        
        for i in cat_page.split(r'<li><a href=')[1:]:  


            # source_link<h2>.*?<a href="   

            source_link = c.scrap('"(.*?)"', i)


            if 'http://www.chinapeace.gov.cn' not in source_link:
                source_link = 'http://www.chinapeace.gov.cn' +  source_link
            if '2021' not in source_link:  # bec of some category
                continue

            print(source_link)
            continue
            
            #logger.info(f'Fetching articles url  {source_link}\n')

    else:  # (h2) present is present 
        
        for i in cat_page.split(r'<h2><a href=')[1:]:
            
            source_link = c.scrap('"(.*?)"', i)


            if 'http://www.chinapeace.gov.cn/' not in source_link:
                source_link = 'http://www.chinapeace.gov.cn' +  source_link
            print(source_link)
            continue
            #logger.info(f'Fetching articles url  {source_link}\n')
            
            


        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))

        #logger.info(f'Fetching {source_link}\n')

        page = c.download_page(source_link)   #here downloading the articles page

        if page.startswith('Unable to fetch'):     
            logger.info(page) 
            unable_to_fetch_article_url += 1
            continue    

        source_headline = c.scrap('<title>(.*?)</title>', page)  #<div class="title">(.(?))<\/div>

        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue

         # Date and time
        pub_date, publish_time = '', ''

        try:

            date_time_str = c.scrap('<meta name="PubDate".*?="(.*?)"/>', page) 
            date_time_str = re.sub('[^\w+]', '', date_time_str)  # to subsitute everything which is not digite
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S')
            ist_date_time = date_time_obj + timedelta(hours = 10,minutes = 30)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]

        except:
            pass
       #         # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')            
            skipped_due_to_date += 1
            continue

    #         # break if date is not today's date
    #         if pub_date != created_on:
    #             break    


    # #         h2 = c.scrap('name="description"\s*content="(.*?)'

    #         # source_content     #div class="content_main">(.*?)<br>/s*<br>     
        source_content= c.scrap('div class="content_main">(.*?)<div class="title_yw">',page)    

        source_content = c.strip_html(source_content)

        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')            
            skipped_due_to_content += 1
            continue



        journalist =c.scrap("'author-name':'(.*?)'",page)

        if not journalist: journalist = 'NA'

        # current date and time 00
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

    #         print('source_link', source_link, '\n')
    #         print('source_headline', source_headline,'\n')
    #         print('pub_date', pub_date,'\n')
    #         print('publish_time', publish_time,'\n')
    #         print('journalist', journalist,'\n')
    #         print('source_content', source_content,'\n')
    #         print('html_content', html_content, '\n')
    #         print('image_urls', image_urls, '\n')



        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

    #         cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')

logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
